In [10]:
import os
import json
import warnings
import numpy as np
import glob
from torch.utils.data import Dataset
warnings.filterwarnings('ignore')

In [11]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    pc = pc / m
    return pc

def drop_txt(data):
    return map(lambda x: x.split('.')[0], data)

In [29]:
root = '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/Pointnet_Pointnet2_pytorch/data/shapenetcore_partanno_segmentation_benchmark_v0_normal'
npoints=2500 
split='train'
class_choice=None
normal_channel=False
    
npoints = npoints
root = root
catfile = os.path.join(root, 'synsetoffset2category.txt')
cat = {}
normal_channel = normal_channel

with open(catfile, 'r') as f:
    for line in f:
        ls = line.strip().split()
        cat[ls[0]] = ls[1]
cat = {k: v for k, v in cat.items()}
classes_original = dict(zip(cat, range(len(cat))))

if not class_choice is  None:
    cat = {k:v for k,v in cat.items() if k in class_choice}

print(cat)

{'Head': '02691156'}


In [30]:
meta = {}

for item, key in cat.items():
    train_ids = [ids for ids in os.listdir(os.path.join(root, key))]
    test_ids = [ids for ids in os.listdir(os.path.join(root, 'test', key))]

vf = np.vectorize(drop_txt)
train_ids = list(drop_txt(train_ids))
test_ids = list(drop_txt(test_ids))
print(train_ids)
print(test_ids)

['5', '2', '1', '6', '3', '7', '4']
['1']


In [31]:
# for item in cat:        
#     meta[item] = []
#     dir_point = os.path.join(root, cat[item])
#     fns = sorted(os.listdir(dir_point))
#     fns = [fn for fn in fns if fn[0:-4] in test_ids]
#     print(fns)

['1.txt']


In [ ]:
for item in cat:        
    meta[item] = []
    dir_point = os.path.join(root, cat[item])
    fns = sorted(os.listdir(dir_point))
    print(fns[0][0:-4])
    if split == 'trainval':
        fns = [fn for fn in fns if ((fn[0:-4] in train_ids) or (fn[0:-4] in val_ids))]
    elif split == 'train':
        fns = [fn for fn in fns if fn[0:-4] in train_ids]
    elif split == 'val':
        fns = [fn for fn in fns if fn[0:-4] in val_ids]
    elif split == 'test':
        fns = [fn for fn in fns if fn[0:-4] in test_ids]
    else:
        print('Unknown split: %s. Exiting..' % (split))
        exit(-1)

    #print(os.path.basename(fns))
    
    for fn in fns:
        token = (os.path.splitext(os.path.basename(fn))[0])
        meta[item].append(os.path.join(dir_point, token + '.txt'))

In [ ]:
datapath = []
for item in cat:
    for fn in meta[item]:
        datapath.append((item, fn))
        
print(datapath)

In [ ]:
classes = {}
for i in cat.keys():
    classes[i] = classes_original[i]
    
print(classes)

In [ ]:
# Mapping from category ('Chair') to a list of int [10,11,12,13] as segmentation labels
seg_classes = {'Head': [0, 1]}

In [ ]:
for cat in sorted(seg_classes.keys()):
    print(cat, seg_classes[cat])

In [ ]:
cache = {}  # from index to (point_set, cls, seg) tuple
cache_size = 20000

In [ ]:
##############################################################

In [ ]:
def __getitem__(self, index):
    if index in self.cache:
        point_set, cls, seg = self.cache[index]
    else:
        fn = self.datapath[index]
        cat = self.datapath[index][0]
        cls = self.classes[cat]
        cls = np.array([cls]).astype(np.int32)
        data = np.loadtxt(fn[1]).astype(np.float32)
        if not self.normal_channel:
            point_set = data[:, 0:3]
        else:
            point_set = data[:, 0:6]
        seg = data[:, -1].astype(np.int32)
        if len(self.cache) < self.cache_size:
            self.cache[index] = (point_set, cls, seg)
    point_set[:, 0:3] = pc_normalize(point_set[:, 0:3])

    choice = np.random.choice(len(seg), self.npoints, replace=True)
    # resample
    point_set = point_set[choice, :]
    seg = seg[choice]

    return point_set, cls, seg

In [ ]:
def __len__(self):
    return len(self.datapath)

In [ ]:
d = PartDataset(root = '../data/shapenetcore_partanno_segmentation_benchmark_v0', class_choice = ['Head'], split='train')
print(len(d))
import time
tic = time.time()
for i in range(len(d)):
    ps, seg = d[i]
    print (np.max(seg), np.min(seg))
print(time.time() - tic)
print(ps.shape, type(ps), seg.shape,type(seg))

d = PartDataset(root = '../data/shapenetcore_partanno_segmentation_benchmark_v0', classification = True)
print(len(d))
ps, cls = d[0]
print(ps.shape, type(ps), cls.shape,type(cls))